In [1]:
import polars as pl

df = pl.read_csv("../data/totalsegmentatormri/meta.csv", separator=";")
df

image_id,age,gender,institute,study_type,manufacturer,scanner_model,slice_thickness,scanning_sequence,repetition_time,echo_time,magnetic_field_strength,source,split
str,f64,str,str,str,str,str,f64,str,f64,f64,f64,str,str
"""s0001""",null,null,"""C""","""unkown""","""ge""","""SIGNA Explorer""",4.4,"""GR""",0.01,0.0,1.5,"""idc""","""train"""
"""s0002""",null,null,"""C""","""unkown""","""ge""","""Signa HDxt""",6.0,"""SE""",3.55,0.15,1.5,"""idc""","""test"""
"""s0003""",null,null,"""C""","""unkown""","""ge""","""Optima MR450w""",5.0,"""EP\SE""",8.0,0.1,1.5,"""idc""","""train"""
"""s0004""",null,null,"""C""","""unkown""","""ge""","""Signa HDxt""",3.0,"""SE""",2.43,0.11,1.5,"""idc""","""train"""
"""s0005""",null,null,"""C""","""unkown""","""ge""","""GENESIS_SIGNA""",8.0,"""SE""",1.53,0.16,1.5,"""idc""","""train"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""s0294""",70.0,"""m""","""B""","""mri thoracic spine""","""siemens""","""Avanto_fit""",3.0,"""GR""",6.68,2.39,1.5,"""usb""","""train"""
"""s0295""",86.0,"""m""","""B""","""mri abdomen""","""siemens""","""Avanto_fit""",3.0,"""GR""",6.68,2.39,1.5,"""usb""","""train"""
"""s0296""",74.0,"""m""","""B""","""mri knee""","""siemens""","""Skyra""",3.0,"""SE""",3270.0,35.0,3.0,"""usb""","""train"""


In [13]:
import torch

a = torch.tensor([[0,1,2,1,0], [1,2,3,2,1], [2,3,4,3,2], [1,2,3,2,1], [0,1,2,1,0]],dtype=torch.float32).unsqueeze(0).unsqueeze(0)
b = torch.nn.Conv2d(1,1,3, bias=False)


b.weight.data.copy_(torch.tensor([[[[2,2.5,2], [2.5,3,2.5], [2,2.5,2]]]]))
b(a)

torch.nn.functional.avg_pool2d(a,(2,2), stride=2)

tensor([[[[1., 2.],
          [2., 3.]]]])